In [60]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import numpy as np

In [1]:
# import os
# os.getcwd()

'/Users/yiweiluo/scientific-debates/cc_stance'

In [3]:
def to_int(x):
    if x == 'neutral' or x == 2:
        return -1
    elif x == 'disagree' or x == 1:
        return 0
    else:
        return 1

In [62]:
BERT_PRED_DIR = 'BERT/bert_preds/'
DATA_DIR = './data_creation/scripts/save/'

# BERT predictions

In [77]:
def do_error_analysis(pred_filename,true_filename):
    # Load and pre-process
    preds = pd.read_csv(BERT_PRED_DIR+pred_filename,sep='\t',header=0,index_col=False)
    true = pd.read_csv(DATA_DIR+true_filename,sep='\t',header=None,index_col=False)
    assert len(preds) == len(true)
    preds['true'] = preds['true'].apply(to_int)
    preds['predicted'] = preds['predicted'].apply(to_int)
    true[1] = true[1].apply(to_int)
    print(true.head(10))
    assert np.sum([(true[1] == preds['true'])]) == len(preds)
    preds['sentence'] = true[0]
    
    bad_preds = preds[preds.true != preds.predicted]
    good_preds = preds[preds.true == preds.predicted]
    opp_preds = bad_preds[bad_preds.true == bad_preds.predicted*-1]
    N_correct = len(good_preds)
    N_wrong = len(bad_preds)
    N_opp = len(opp_preds)
    total_N = len(preds)
    print('Num correct: {},\tNum wrong: {},\tNum opposite: {}\n'.format(N_correct,
                                                                N_wrong,
                                                                N_opp))
    print('Percent correct: {},\tPercent wrong: {},\tPercent opposite: {}\n'\
          .format(round(N_correct/len(preds),3),
                 round(N_wrong/len(preds),3),
                 round(N_opp/len(preds),3)))
    
    prop_opp_from_agrees = round(len(opp_preds[opp_preds.true == 1])/len(opp_preds)*100,2)
    print('Of the opposite preds, {}% were true agrees and {}% were true disagrees.\n'\
         .format(prop_opp_from_agrees,100-prop_opp_from_agrees))
    
    false_neutrals = bad_preds[bad_preds.predicted == 0]
    false_agrees = bad_preds[bad_preds.predicted == 1]
    false_disagrees = bad_preds[bad_preds.predicted == -1]
    true_neutrals = good_preds[good_preds.predicted == 0]
    true_agrees = good_preds[good_preds.predicted == 1]
    true_disagrees = good_preds[good_preds.predicted == -1]
    
    print('Proportion of false agrees: {} (out of {} true agrees)'.\
          format(round(len(false_agrees)/len(preds),3),
                len(true_agrees)))
    print('Distribution of true values of false agrees:\n', 
          false_agrees.true.value_counts())
    
    print('Proportion of false disagrees: {} (out of {} true disagrees)'.\
          format(round(len(false_disagrees)/len(preds),3),
                len(true_disagrees)))
    print('Distribution of true values of false disagrees:\n', 
          false_disagrees.true.value_counts())
    
    print('Proportion of false neutrals: {} (out of {} true neutrals)'.\
          format(round(len(false_neutrals)/len(preds),3),
                len(true_neutrals)))
    print('Distribution of true values of false neutrals:\n', 
          false_neutrals.true.value_counts())
    
    return preds,opp_preds

In [80]:
preds_dev,opp_preds_dev = do_error_analysis('downsampled_mturk_dev_preds.tsv',
                 'downsampled_mturk/dev.tsv')

                                                                                                                                                                                                                                                 0  \
0  The effects of decisions like these will be felt for generations as global warming leads to more powerful storms and higher risk of damage to property.                                                                                           
1  Hawaii is committed to environmental stewardship, and we look forward to working with other states to fight global climate change.                                                                                                                
2  The global south in particular is both more vulnerable to the effects of climate change and less equipped to deal with those effects.                                                                                                             
3  They would lo

In [79]:
# Train on downsampled MTurk; evaluate on dev MTurk
opp_preds

,true,predicted,sentence
31,1,-1,They are too pessimistic about the chances of limiting global warming.
45,1,-1,Its appropriate to call such storms global warming type events.
167,-1,1,The magnitude of the dryness is well beyond what global warming would be able to do so far.
171,-1,1,Global warming is occurring more slowly than computer models forecast a decade ago.
185,-1,1,Global warming can be tackled without changing entrenched patterns of production and consumption in industrialized societies.
188,-1,1,"Ultimately, global warming has become the sole focus of so much environmental discussion that it risks eclipsing much more pressing and demonstrable environmental problems."
189,-1,1,Mitch McConnell and the Republican Party are going to boldly confront climate change and create millions of jobs in the process.
207,-1,1,By now there should have been global warming equivalent to 2.8 degrees / century.
210,-1,1,Policies designed to curb global warming would hurt the economy.
225,-1,1,"All that our world leaders need is a little more common sense, in order to avoid the nonsense of seeking a globally-binding treaty to reduce CO2 emissions."


In [81]:
preds_test,opp_preds_test = do_error_analysis('downsampled_mturk_test_preds.tsv',
                 'downsampled_mturk/test.tsv')

                                                                                                                                                                                                                                                                                                                               0  \
0  The ridiculously warm temperatures in the Arctic during October and November this year this year are off the charts over our 68 years of measurements.                                                                                                                                                                          
1  No one can say they are happy about climate change.                                                                                                                                                                                                                                                                             
2  Earlier this year painted

In [82]:
opp_preds_test

,true,predicted,sentence
0,1,-1,The ridiculously warm temperatures in the Arctic during October and November this year this year are off the charts over our 68 years of measurements.
5,1,-1,Living like climate change is real and that we can do something about it are signals to others.
10,1,-1,The low number is derived from a method that did not take account of the way global warming had sped up since the 1970s.
37,1,-1,"It does not include a discussion on climate change, which is an outrage."
87,1,-1,No use in denying global warming exists.
89,1,-1,"Global warming is the tide coming in, and El Nio is the waves."
102,1,-1,"The beans contains some powerful refutations of a number of other global warming denier talking points, as well as some ingenious sequences that explain the planetary-scale significance of climate change."
115,1,-1,The missed deadline underscores the disdain with which the Trump administration views the worldwide effort to rein in global warming.
116,1,-1,"Today 's [ Green New Deal ] vote is a partisan stunt to side-step needed debate on climate action, and give Republicans cover to put oil lobby checks over our kids."
146,1,-1,"Global warming is real, because the rise of terrorism tracks with their adjusted climate data."


BERT trained on MTurk only, no downsampling:

More of the opposite predictions are coming from true "disagrees" than "agrees".

In [43]:
len(opp_preds[opp_preds.true == 1])/len(opp_preds)

0.18181818181818182

In [44]:
len(opp_preds[opp_preds.true == -1])/len(opp_preds)

0.8181818181818182

In [58]:
opp_preds

,true,predicted,sentence
31,1,-1,They are too pessimistic about the chances of limiting global warming.
45,1,-1,Its appropriate to call such storms global warming type events.
167,-1,1,The magnitude of the dryness is well beyond what global warming would be able to do so far.
171,-1,1,Global warming is occurring more slowly than computer models forecast a decade ago.
185,-1,1,Global warming can be tackled without changing entrenched patterns of production and consumption in industrialized societies.
188,-1,1,"Ultimately, global warming has become the sole focus of so much environmental discussion that it risks eclipsing much more pressing and demonstrable environmental problems."
189,-1,1,Mitch McConnell and the Republican Party are going to boldly confront climate change and create millions of jobs in the process.
207,-1,1,By now there should have been global warming equivalent to 2.8 degrees / century.
210,-1,1,Policies designed to curb global warming would hurt the economy.
225,-1,1,"All that our world leaders need is a little more common sense, in order to avoid the nonsense of seeking a globally-binding treaty to reduce CO2 emissions."


In [173]:
false_neutrals = bad_preds[bad_preds.predicted == 0]
false_agrees = bad_preds[bad_preds.predicted == 1]
false_disagrees = bad_preds[bad_preds.predicted == -1]
true_neutrals = good_preds[good_preds.predicted == 0]
true_agrees = good_preds[good_preds.predicted == 1]
true_disagrees = good_preds[good_preds.predicted == -1]

In [175]:
len(false_agrees)/len(mturk_only_preds)

0.15030674846625766

In [176]:
len(true_agrees)

84

In [177]:
false_agrees.true.value_counts()

 0    28
-1    21
Name: true, dtype: int64

In [179]:
len(false_disagrees)/len(mturk_only_preds)

0.08895705521472393

In [180]:
len(true_disagrees)

27

In [181]:
false_disagrees.true.value_counts()

0    15
1    14
Name: true, dtype: int64

In [182]:
len(false_neutrals)/len(mturk_only_preds)

0.13496932515337423

In [183]:
len(true_neutrals)

93

In [184]:
false_neutrals.true.value_counts()

 1    35
-1    9 
Name: true, dtype: int64

BERT Trained on all data:

In [131]:
len(opp_preds[opp_preds.true == 1])/len(opp_preds)

0.25

In [132]:
len(opp_preds[opp_preds.true == -1])/len(opp_preds)

0.75

In [119]:
opp_preds

,true,predicted,sentence
13,-1,1,This a sneaky way to cover up the reality that global warming is a sham.
17,-1,1,An increase in C02 emissions from fossil fuels will be a great benefit to plant and animal life on Earth.
42,-1,1,Global warming idiots out of danger.
49,-1,1,"The world may have already begun to cool in response to a weakening Sun, a phenomenon far more dangerous than any possible human-induced warming."
50,-1,1,Global warming helps plants — and thus also helps humans and animals .
58,-1,1,There is a rush to judgment that every time something happens it’s because of global warming.
60,-1,1,"For example, stressing the idea that human activity is behind global warming can be counterproductive."
86,-1,1,Global warming is a relatively distant threat.
88,-1,1,Satellite data disputes global warming.
98,-1,1,Even in liberal states doubt global warming is caused mostly by human carbon emissions.


In [120]:
false_neutrals = bad_preds[bad_preds.predicted == 0]
false_agrees = bad_preds[bad_preds.predicted == 1]
false_disagrees = bad_preds[bad_preds.predicted == -1]
true_neutrals = good_preds[good_preds.predicted == 0]
true_agrees = good_preds[good_preds.predicted == 1]
true_disagrees = good_preds[good_preds.predicted == -1]

In [121]:
len(false_agrees)/407.

0.3857493857493858

In [122]:
len(true_agrees)

132

In [123]:
false_agrees.true.value_counts()

 0    127
-1    30 
Name: true, dtype: int64

In [124]:
len(false_disagrees)/407

0.07616707616707617

In [125]:
len(true_disagrees)

30

In [126]:
false_disagrees.true.value_counts()

0    21
1    10
Name: true, dtype: int64

In [127]:
len(false_neutrals)/407

0.04668304668304668

In [129]:
len(true_neutrals)

38

In [128]:
false_neutrals.true.value_counts()

 1    13
-1    6 
Name: true, dtype: int64

# GPT(?) predictions 

In [51]:
bad_preds['true_stance'] = bad_preds['true_stance'].apply(to_int)
bad_preds['pred_stance'] = bad_preds['pred_stance'].apply(to_int)
good_preds['true_stance'] = good_preds['true_stance'].apply(to_int)
good_preds['pred_stance'] = good_preds['pred_stance'].apply(to_int)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [46]:
bad_preds = preds[preds.true_stance != preds.pred_stance]
good_preds = preds[preds.true_stance == preds.pred_stance]
opp_preds = bad_preds[bad_preds.true_stance == bad_preds.pred_stance*-1]

In [47]:
bad_preds.shape

(111, 4)

In [48]:
good_preds.shape

(53, 4)

In [54]:
opp_preds.shape

(22, 4)

In [55]:
false_neutrals = bad_preds[bad_preds.pred_stance == 0]
false_agrees = bad_preds[bad_preds.pred_stance == 1]
false_disagrees = bad_preds[bad_preds.pred_stance == -1]
true_neutrals = good_preds[good_preds.pred_stance == 0]
true_agrees = good_preds[good_preds.pred_stance == 1]
true_disagrees = good_preds[good_preds.pred_stance == -1]

27% of predictions were false "agrees". More "neutrals" than "disagrees" were falsely predicted.
There were 19 true agrees.

In [31]:
len(false_agrees)/164

0.27439024390243905

In [41]:
len(true_agrees)

19

In [33]:
false_agrees.true_stance.value_counts()

 0    33
-1    12
Name: true_stance, dtype: int64

13% of predictions were false "disagrees". About equal numbers of "agrees" and "neutrals" were falsely predicted. There were 7 true disagrees. 

In [57]:
len(false_disagrees)/164

0.12804878048780488

In [58]:
len(true_disagrees)

7

In [59]:
false_disagrees.true_stance.value_counts()

0    11
1    10
Name: true_stance, dtype: int64

27% of predictions were false neutrals. More "agrees" than "disagrees" were getting predicted to be "neutral".

In [27]:
len(false_neutrals)/164.

0.27439024390243905

In [28]:
false_neutrals.true_stance.value_counts()

 1    29
-1    16
Name: true_stance, dtype: int64

13% of predictions were the opposite of the correct label (e.g., "agree" was predicted "disagree" or vice versa.

In [61]:
len(opp_preds)/164.

0.13414634146341464

Of this 13%, about half are mistaken "agrees" (and the other 55% mistaken "disagrees").

In [24]:
len(opp_preds[opp_preds.true_stance == 1])/len(opp_preds)

0.45454545454545453

In [65]:
opp_preds

,type,sentence,true_stance,pred_stance
4,mturk,Global warming isn't real Leo.,-1,1
6,mturk,"A combination of factors are at work, including a natural El Nino system, man-made global warming and a dash of random weather.",-1,1
17,mturk,The paper in no way diminishes the extensive body of observations that global warming is happening and that it is largely due to human activity.,1,-1
23,mturk,"Carbon dioxide fertilizes plants, and emissions from fossil fuels have already had a hugely beneficial effect on crops, increasing yields by at least 10-15 percent.",-1,1
34,mturk,Global warming is having an impact now.,1,-1
36,mturk,Man-made climate change isn't solely to blame for tropical storms.,-1,1
37,mturk,Fossil fuel investments are pretty good bets regardless of the political climate.,-1,1
58,mturk,The claim made in the 2007 report by the Intergovernmental Panel on Climate Change (IPCC) is unrealistic and the large Himalayan glaciers could not melt in a few decades.,-1,1
59,mturk,Climate change would be used as an excuse to expand `big government' and lead to a loss of personal freedom.,-1,1
61,mturk,Global warming is real.,1,-1


In [63]:
list(opp_preds.sentence)

["Global warming isn't real Leo.",
 'A combination of factors are at work, including a natural El Nino system, man-made global warming and a dash of random weather.',
 'The paper in no way diminishes the extensive body of observations that global warming is happening and that it is largely due to human activity.',
 'Carbon dioxide fertilizes plants, and emissions from fossil fuels have already had a hugely beneficial effect on crops, increasing yields by at least 10-15 percent.',
 'Global warming is having an impact now.',
 "Man-made climate change isn't solely to blame for tropical storms.",
 'Fossil fuel investments are pretty good bets regardless of the political climate.',
 'The claim made in the 2007 report by the Intergovernmental Panel on Climate Change (IPCC) is unrealistic and the large Himalayan glaciers could not melt in a few decades.',
 "Climate change would be used as an excuse to expand `big government' and lead to a loss of personal freedom.",
 'Global warming is real.'